In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import date, time, datetime

Sections :: 
Introduction
Data Exploration
Explanation of Code
Results 
Analysis

Here we're importing our dataset, we use the sessions data later on for our 

In [2]:
train_users = pd.read_csv("proj3_data/train_users.csv",parse_dates=['timestamp_first_active','date_account_created','date_first_booking'])
test_users = pd.read_csv("proj3_data/test_users.csv",parse_dates=['timestamp_first_active','date_account_created','date_first_booking'])
sessions = pd.read_csv("proj3_data/sessions.csv")

In [3]:

Y = train_users['country_destination'] # these are our labels
train_users = train_users.drop('country_destination', axis =1)



In [4]:
train_users.columns # exploratory 

Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser'],
      dtype='object')

In [5]:
df_all = pd.concat((train_users, test_users), axis = 0, ignore_index= True)

In [6]:
from sklearn.preprocessing import MinMaxScaler

# Splitting date time data for date account created
df_all['dac_year'] = df_all.date_account_created.dt.year
df_all['dac_month'] = df_all.date_account_created.dt.month
df_all['dac_day'] = df_all.date_account_created.dt.day

# Splitting date time data for time first active
df_all['tfa_year'] = df_all.timestamp_first_active.dt.year
df_all['tfa_month'] = df_all.timestamp_first_active.dt.month
df_all['tfa_day'] = df_all.timestamp_first_active.dt.day

df_all.drop('date_account_created',1, inplace=True)
df_all.drop('timestamp_first_active',1, inplace=True)



In [7]:

#df_all.loc[df_all.age > 100, 'age'] = np.nan
#df_all.loc[df_all.age < 18, 'age'] = np.nan

age_values = df_all.age.values
df_all['age'] = np.where(age_values>1000, np.random.randint(28, 43), age_values)
df_all['age'] = df_all['age'].fillna(np.random.randint(28, 43))



In [8]:
by_signup_device = df_all.groupby(['signup_method', 'first_device_type'])
def impute_median(series):
    return series.fillna(series.median())
df_all.age = by_signup_device['age'].transform(impute_median)

In [9]:
tracked = []

for i in df_all['first_affiliate_tracked']:
    if i == "untracked" or i == "":
        isTracked = 0
    else:
        isTracked = 1
    tracked.append(isTracked)

df_all['is_first_affiliate_tracked'] = tracked

In [10]:
#seconds = sessions.groupby('id', as_index=False).agg({"secs_elapsed": "sum"})
#seconds.head()
#df_all = pd.merge(df_all, seconds, on="id", how="left")
#df_all['secs_elapsed'] = df_all['secs_elapsed'].fillna(0)

In [11]:
method = pd.get_dummies(df_all[["signup_method"]])
affch = pd.get_dummies(df_all[["affiliate_channel"]])
affprov = pd.get_dummies(df_all[["affiliate_provider"]])
firstdevice = pd.get_dummies(df_all[["first_device_type"]])
signupFlow = pd.get_dummies(df_all[["signup_flow"]].astype(str))
signup = pd.get_dummies(df_all[["signup_app"]])
genderdum = pd.get_dummies(df_all[["gender"]])
langdum = pd.get_dummies(df_all[["language"]])
browser = pd.get_dummies(df_all[["first_browser"]])

df2 = df_all

df2 = pd.concat([df2.reset_index(drop=True),method.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),affch.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),affprov.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),firstdevice.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),signupFlow.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),genderdum.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),signup.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),langdum.reset_index(drop=True)], axis=1)
df2 = pd.concat([df2.reset_index(drop=True),browser.reset_index(drop=True)], axis=1)

In [12]:
df2 = df2.drop(['id',  'date_first_booking','gender', 
          'signup_method',  'affiliate_channel', 'affiliate_provider', 'first_device_type', 
          'first_browser', 'signup_app', 'first_browser'], axis=1)



In [13]:
df2 = df2.drop("language", axis=1)

In [14]:
df2 = df2.drop('signup_flow', axis=1)
df2 = df2.drop(['first_affiliate_tracked'], axis=1)

In [15]:
id_test = test_users['id']

In [16]:
print(len(Y.unique()))
print(Y.unique())

12
['NDF' 'US' 'other' 'FR' 'CA' 'GB' 'ES' 'IT' 'PT' 'NL' 'DE' 'AU']


In [34]:
print(df2.shape)
print(len(Y))
print(train.shape)
print(df_all.shape)

(275547, 154)
213451
(213451, 154)
(275547, 20)


In [52]:
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection



#Y = train_users['country_destination'] # these are our labels
#train_users = train_users.drop('country_destination', axis =1)



train = df2.iloc[0:len(train_users),:]
test = df2.iloc[len(train_users):,:]


X_train, X_test, y_train, y_test = model_selection.train_test_split(
    train, Y, test_size=0.33, random_state=42)


#train_users_n = train_users.shape[0] # comment
#X_train = df2.values[:train_users_n] # comment
le = LabelEncoder() # don't uncommment
#y_train = le.fit_transform(Y)

y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)   


#X_test = df2.values[train_users_n:]


def generate_answer(y_pred, classifer_name):
    #Taking the 5 classes with highest probabilities
    ids = []  #list of ids
    cts = []  #list of countries
    for i in range(len(id_test)):
        idx = id_test[i]
        ids += [idx] * 5
        cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()
    
    sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
    sub.to_csv(classifer_name+'.csv',index=False)
    return sub

In [38]:
len(y_train)
len(X_train)
len(X_test)

70439

In [50]:
from xgboost.sklearn import XGBClassifier

xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict_proba(X_test)


In [19]:
len(y_pred_xgb)

70439

In [28]:
X_train.shape

(143012, 154)

In [20]:
generate_answer(y_pred_xgb, 'XGB')

,id,country
0,5uwns89zht,US
1,5uwns89zht,NDF
2,5uwns89zht,other
3,5uwns89zht,FR
4,5uwns89zht,IT
...,...,...
310475,9uqfg8txu3,US
310476,9uqfg8txu3,NDF
310477,9uqfg8txu3,other
310478,9uqfg8txu3,IT


In [54]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
ids = []  #list of ids
cts = []  #list of countries
y_predicted = y_pred_xgb
for i in range(len(y_pred_xgb)):
    #idx = id_test[i]
    #ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_predicted[i])[::-1])[:1].tolist()

#scores = cross_val_score(xgb, X_train, y_train, cv = 10)
#multilabel_confusion_matrix(y_test, y_pred_xgb)

In [45]:
print(len(cts))
print(len(y_test))
print(len(y_pred_xgb))
print(y_pred_xgb[0])
print(le.inverse_transform(np.argsort(y_pred_xgb[0])[::-1])[:1].tolist())


352195
70439
70439
[0.00585135 0.01071016 0.00408473 0.01548126 0.03471473 0.01679099
 0.02006238 0.2828004  0.0054968  0.00232967 0.52838033 0.07329721]
['US']


In [46]:
70439 * 5

352195

In [49]:
print(classification_report(y_test, cts))

              precision    recall  f1-score   support

          AU       0.00      0.00      0.00       182
          CA       0.00      0.00      0.00       456
          DE       0.00      0.00      0.00       358
          ES       0.00      0.00      0.00       743
          FR       0.00      0.00      0.00      1645
          GB       0.00      0.00      0.00       837
          IT       0.00      0.00      0.00       952
         NDF       0.69      0.86      0.76     40905
          NL       0.00      0.00      0.00       241
          PT       0.00      0.00      0.00        86
          US       0.49      0.46      0.48     20714
       other       0.00      0.00      0.00      3320

    accuracy                           0.63     70439
   macro avg       0.10      0.11      0.10     70439
weighted avg       0.54      0.63      0.58     70439



In [76]:
from sklearn.metrics import ndcg_score
#print(y_test)
#print(y_pred_xgb)
#print(len(y_test))
#print(len(y_pred_xgb))


print(y_test[0])
print(y_pred_xgb[0])
print(len(y_pred_xgb[0]))



testArray = [[0,0,0,1,0,0,0,0,0,0,0,0]]
print(len(testArray))
testArray2 = [y_pred_xgb[0]]



#print(cts[0])
ndcg_score(testArray, testArray2)

3
[0.00585135 0.01071016 0.00408473 0.01548126 0.03471473 0.01679099
 0.02006238 0.2828004  0.0054968  0.00232967 0.52838033 0.07329721]
12
1


0.3333333333333335

In [ ]:


# Precision- > measure of result relevancy
# Recall -> measure of how many relevant results were reutnred


